<a href="https://colab.research.google.com/github/martintmv-git/RB-IBDM/blob/main/Detectron2/Detectron2_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="align-center">
  <a href="https://www.fontys.nl/"><img src="https://www.fontys.nl/static/design/FA845701-BD71-466E-9B3D-38580DFAD5B4-fsm/images/logo-inverted@2x.png" height="75"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.has.nl/"><img src="https://i.imgur.com/ZxkugVW.png" height="75"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.has.nl/onderzoek/lectoraten/lectoraat-innovatieve-biomonitoring/"><img src="https://i.imgur.com/oH3VJpE.png" height="75"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.naturalis.nl/"><img src="https://i.imgur.com/mAHW7XQ.png" height="75"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://www.arise-biodiversity.nl/"><img src="https://i.imgur.com/j6gBpqT.png" height="75"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://faunabit.eu/"><img src="https://i.imgur.com/HxqzRYg.png" height="70"></a>
  <img src="https://i.imgur.com/zyfbV3r.png" width="20">
  <a href="https://diopsis.eu/"><img src="https://i.imgur.com/NHZ8e1b.png" height="75"></a>
</div>

# **How to Use the Detectron2 Object Detection on New Images**

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/martintmv-git/RB-IBDM) <a href="https://github.com/facebookresearch/detectron2"><img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="120"></a>

<b>This notebook contains two main parts:
1. Environment setup
2. Inference

You need to run the cells in the forst part in order to get everything set up and ready to use and then pass either the path to the image to analyze or the path to the folder containing multiple images to the `detect` function for inference.</b>

# Environment setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install --quiet {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import pandas as pd
import os, json, cv2, random
from datetime import datetime
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode

In [ ]:
cfg = get_cfg()
cfg.merge_from_file('/content/drive/MyDrive/Fontys/Fontys_Sem7/insect_detection/Detectron2 - Faster-RCNN/results5/config.yaml')
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Fontys/Fontys_Sem7/insect_detection/Detectron2 - Faster-RCNN/results5/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
predictor = DefaultPredictor(cfg)

In [ ]:
thing_classes = {'thing_classes':['Acrididae',
                         'Agapeta',
                         'Agapeta hamana',
                         'Animalia',
                         'Anisopodidae',
                         'Aphididae',
                         'Apidae',
                         'Arachnida',
                         'Araneae',
                         'Arctiidae',
                         'Auchenorrhyncha indet.',
                         'Baetidae',
                         'Cabera',
                         'Caenidae',
                         'Carabidae',
                         'Cecidomyiidae',
                         'Ceratopogonidae',
                         'Cercopidae',
                         'Chironomidae',
                         'Chrysomelidae',
                         'Chrysopidae',
                         'Chrysoteuchia culmella',
                         'Cicadellidae',
                         'Coccinellidae',
                         'Coleophoridae',
                         'Coleoptera',
                         'Collembola',
                         'Corixidae',
                         'Crambidae',
                         'Culicidae',
                         'Curculionidae',
                         'Dermaptera',
                         'Diptera',
                         'Eilema',
                         'Empididae',
                         'Ephemeroptera',
                         'Erebidae',
                         'Fanniidae',
                         'Formicidae',
                         'Gastropoda',
                         'Gelechiidae',
                         'Geometridae',
                         'Hemiptera',
                         'Hydroptilidae',
                         'Hymenoptera',
                         'Ichneumonidae',
                         'Idaea',
                         'Insecta',
                         'Lepidoptera',
                         'Leptoceridae',
                         'Limoniidae',
                         'Lomaspilis marginata',
                         'Miridae',
                         'Mycetophilidae',
                         'Nepticulidae',
                         'Neuroptera',
                         'Noctuidae',
                         'Notodontidae',
                         'Object',
                         'Opiliones',
                         'Orthoptera',
                         'Panorpa germanica',
                         'Panorpa vulgaris',
                         'Parasitica indet.',
                         'Plutellidae',
                         'Psocodea',
                         'Psychodidae',
                         'Pterophoridae',
                         'Pyralidae',
                         'Pyrausta',
                         'Sepsidae',
                         'Spilosoma',
                         'Staphylinidae',
                         'Stratiomyidae',
                         'Syrphidae',
                         'Tettigoniidae',
                         'Tipulidae',
                         'Tomoceridae',
                         'Tortricidae',
                         'Trichoptera',
                         'Triodia sylvina',
                         'Yponomeuta',
                         'Yponomeutidae']}

In [ ]:
def single_detect(image_path, save_csv = True, save_image = True):
  path, file_name = os.path.split(image_path)
  save_path = os.path.join(path, file_name.replace('.jpg', '_detections.jpg'))

  year = file_name[:4]
  month = file_name[4:6]
  day = file_name[6:8]
  hour = file_name[8:10]
  minute = file_name[10:12]
  second = file_name[12:14]

  # Read image
  im = cv2.imread(image_path)
  H, W, _ = im.shape

  # Predict using model
  outputs = predictor(im)

  if save_csv:
    # Save detections to CSV file
    tracking_results = []
    out_dict = {}
    instances = outputs["instances"].to("cpu")
    num_instance = len(instances)
    if num_instance == 0:
        out_dict['filename'] = file_name
        out_dict['date'] = f'{year}-{month}-{day}'
        out_dict['time'] = f'{hour}:{minute}:{second}'
        out_dict['date_time'] = f'{day}/{month}/{year} {hour}:{minute}'
        out_dict['name_predicted'] = None
        out_dict['probability'] = None
        out_dict['body_length_mm'] = None
        out_dict['biomass_mg'] = None
        out_dict['region'] = None
        tracking_results.append(out_dict)
        out_dict = {}
    else:
        boxes = instances.pred_boxes.tensor.numpy()
        boxes = boxes.tolist()
        scores = instances.scores.tolist()
        classes = instances.pred_classes.tolist()

        for k in range(num_instance):
            box = boxes[k]
            out_dict['filename'] = file_name
            out_dict['date'] = f'{year}-{month}-{day}'
            out_dict['time'] = f'{hour}:{minute}:{second}'
            out_dict['date_time'] = f'{day}/{month}/{year} {hour}:{minute}'
            out_dict['name_predicted'] = thing_classes['thing_classes'][classes[k]]
            out_dict['probability'] = scores[k]
            out_dict['body_length_mm'] = None
            out_dict['biomass_mg'] = None
            out_dict['region'] = f'x1={box[0]/W};x2={box[2]/W};y1={box[1]/H};y2={box[3]/H}'

            # out_dict['x1'] = box[0]
            # out_dict['y1'] = box[1]
            # out_dict['x2'] = box[2]
            # out_dict['y2'] = box[3]
            tracking_results.append(out_dict)
            out_dict = {}

    df = pd.DataFrame(tracking_results)
    df.to_csv(os.path.join(path, file_name.replace('.jpg', '.csv')), index=False)

  # Visualize detections
  v = Visualizer(im[:, :, ::-1], thing_classes)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

  if save_image:
    # Save annotated image
    cv2.imwrite(save_path, out.get_image()[:, :, ::-1])

  # Display image
  cv2_imshow(out.get_image()[:, :, ::-1])

  return df

def batch_detect(path, save_csv = True, save_image = True):
  save_path = os.path.join(path, 'detectron_output')
  # save_path = path
  if os.path.exists(save_path) == False:
    os.mkdir(save_path)

  tracking_results = []

  for file in os.listdir(path):
    if file.endswith('.jpg'):
      year = file[:4]
      month = file[4:6]
      day = file[6:8]
      hour = file[8:10]
      minute = file[10:12]
      second = file[12:14]

      # Read image
      im = cv2.imread(os.path.join(path, file))
      H, W, _ = im.shape

      # Predict using model
      outputs = predictor(im)

      if save_csv:
        # Save detections to CSV file
        out_dict = {}
        instances = outputs["instances"].to("cpu")
        num_instance = len(instances)
        if num_instance == 0:
            out_dict['filename'] = file
            out_dict['date'] = f'{year}-{month}-{day}'
            out_dict['time'] = f'{hour}:{minute}:{second}'
            out_dict['date_time'] = f'{day}/{month}/{year} {hour}:{minute}'
            out_dict['name_predicted'] = None
            out_dict['probability'] = None
            out_dict['body_length_mm'] = None
            out_dict['biomass_mg'] = None
            out_dict['region'] = None
            tracking_results.append(out_dict)
            out_dict = {}
        else:
            boxes = instances.pred_boxes.tensor.numpy()
            boxes = boxes.tolist()
            scores = instances.scores.tolist()
            classes = instances.pred_classes.tolist()

            for k in range(num_instance):
                box = boxes[k]
                out_dict['filename'] = file
                out_dict['date'] = f'{year}-{month}-{day}'
                out_dict['time'] = f'{hour}:{minute}:{second}'
                out_dict['date_time'] = f'{day}/{month}/{year} {hour}:{minute}'
                out_dict['name_predicted'] = thing_classes['thing_classes'][classes[k]]
                out_dict['probability'] = scores[k]
                out_dict['body_length_mm'] = None
                out_dict['biomass_mg'] = None
                out_dict['region'] = f'x1={box[0]/W};x2={box[2]/W};y1={box[1]/H};y2={box[3]/H}'
                tracking_results.append(out_dict)
                out_dict = {}

      if save_image:
        # Visualize detections
        v = Visualizer(im[:, :, ::-1], thing_classes)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        # Save annotated image
        cv2.imwrite(os.path.join(save_path, file.replace('.jpg', '_detections.jpg')), out.get_image()[:, :, ::-1])

  if save_csv:
    df = pd.DataFrame(tracking_results)
    df.to_csv(os.path.join(save_path, 'results.csv'), index=False)

def detect(path):
  if os.path.isdir(path):
    batch_detect(path)
  else:
    single_detect(path)

# Inference

## Single image

In [ ]:
# Select image file
image_path = '/content/drive/MyDrive/Fontys/Fontys_Sem7/insect_detection/Detectron2 - Faster-RCNN/test_images/20230715012944.jpg'
detect(image_path)

## Batch

In [ ]:
# Select folder containing images
folder_path = '/content/drive/MyDrive/Fontys/Fontys_Sem7/insect_detection/Detectron2 - Faster-RCNN/test_images'
detect(folder_path)